In [2]:
import pandas as pd

In [12]:
song_df=pd.read_table('10000.txt',header=None)

In [15]:
song_df.columns=['user_id','song_id','count']

In [21]:
print (len(song_df['user_id']))

2000000


In [22]:
song_info_df=pd.read_csv("https://static.turi.com/datasets/millionsong/song_data.csv")

In [27]:
print (song_info_df.head())

              song_id              title  \
0  SOQMMHC12AB0180CB8       Silent Night   
1  SOVFVAK12A8C1350D9        Tanssi vaan   
2  SOGTUKN12AB017F4F1  No One Could Ever   
3  SOBNYVR12A8C13558C      Si Vos Querés   
4  SOHSBXH12A8C13B0DF   Tangle Of Aspens   

                                release       artist_name  year  
0                 Monster Ballads X-Mas  Faster Pussy cat  2003  
1                           Karkuteillä  Karkkiautomaatti  1995  
2                                Butter    Hudson Mohawke  2006  
3                               De Culo       Yerba Brava  2003  
4  Rene Ablaze Presents Winter Sessions        Der Mystic     0  
